# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, the main goal is to identify lanes on the road. While the task is easy for people, it's a complicated problem for an autonous system. I'll use Computer vision algorithms to solve the problem.

All the details are explained in the README.md

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os
from typing import List, Tuple
from __future__ import annotations
from collections import deque
from scipy import stats
%matplotlib inline

## Helper Functions

Below are some helper functions to help get us started. They should look familiar from the lesson!

In [2]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

## Build a Lane Finding Pipeline



In [3]:
class LaneLinesDetector:
    """
    The class implements the lane lines detector. The detector uses internally
    the next pipeline:
        - Applies Grayscale Transform
        - Applies Gaussian Noise
        - Detects Canny Edges
        - Filters the uninteresting region out
        - Applies Hough Transform to detect line segments
        - Extrapolate lane lines from line segments
        - Stabilize lane lines
        - Adds overlayed lane lines to the original image
    """
    
    def __init__(self,
                 region: List[Tuple[int, int]],
                 stabilization_history_size: int = 10,
                 gaussian_blur_kernel: int = 5,
                 canny_low_threshold: int = 50, canny_high_threshold: int = 150,                 
                 hough_rho: int = 1, hough_theta: float = np.pi/180, hough_threshold: int = 15,
                 hough_min_line_length: int = 20, hough_max_line_gap: int = 10,
                 min_slope: float = 0.1,
                 approximation_slope_diff: float = 0.1, approximation_intercept_diff: float = 50.0):
        """
        Initializes the detector.
        """
        self.left_history = deque(maxlen=stabilization_history_size)
        self.right_history = deque(maxlen=stabilization_history_size)

        self.gaussian_blur_kernel = gaussian_blur_kernel
        
        self.canny_low_threshold = canny_low_threshold
        self.canny_high_threshold = canny_high_threshold
        
        self.region = region

        self.hough_rho = hough_rho
        self.hough_theta = hough_theta
        self.hough_threshold = hough_threshold
        self.hough_min_line_length= hough_min_line_length
        self.hough_max_line_gap = hough_max_line_gap

        self.min_slope = min_slope
        self.approximation_slope_diff = approximation_slope_diff
        self.approximation_intercept_diff = approximation_intercept_diff

    def pipeline(self, img):
        """ Processes the image """
        self.img = img        
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        blur = gaussian_blur(gray, self.gaussian_blur_kernel)
        edges = canny(blur, self.canny_low_threshold, self.canny_high_threshold)        
        masked = region_of_interest(edges, np.array([self.region], dtype=np.int32))        
        lines = cv2.HoughLinesP(masked, self.hough_rho, self.hough_theta, self.hough_threshold, np.array([]),
                                minLineLength=self.hough_min_line_length, maxLineGap=self.hough_max_line_gap)
        left, right = self.split_lines(lines)        
        line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
        draw_lines(line_img, self.extrapolate(left, self.left_history), thickness=20)
        draw_lines(line_img, self.extrapolate(right, self.right_history), thickness=20)
        return weighted_img(img, line_img)        

    def split_lines(self, lines):
        """
        Splits lines onto two groups.
        The group represents the set of lines either for the left line, or for the right line.
        """
        left_lines, right_lines = [], []
        mid_x = self.img.shape[1] // 2
        for line in lines:
            for x1, y1, x2, y2 in line:
                dx, dy = x2 - x1, y2 - y1
                if dx == 0 or dy == 0:
                    continue
                    
                slope = dy / dx
                # Filter out uninteresting lines (almost horizontal)
                if abs(slope) <= self.min_slope:
                    continue
                    
                if slope < 0 and x1 < mid_x and x2 < mid_x:
                    left_lines.append([[x1, y1, x2, y2]])
                elif x1 >= mid_x and x2 >= mid_x:
                    right_lines.append([[x1, y1, x2, y2]])
                    
        return left_lines, right_lines

    def mean(self, history):
        """ Helper to find the mean over the history """
        return [0, 0] if not history else np.mean(history, axis=0)

    def approximate(self, k, b, history):
        """ Approximates coefficients according with the history """
        if not history:
            history.append([k, b])
            return k, b

        mean = self.mean(history)
        kd = abs(k - mean[0])
        bd = abs(b - mean[1])
        
        if kd > self.approximation_slope_diff or bd > self.approximation_intercept_diff:
            return mean

        history.append([k, b])
        return self.mean(history)

    def extrapolate(self, lines, history):
        """ Extrapolate the set of lines to the one line """
        x, y = [], []
        
        for line in lines:
            for x1, y1, x2, y2 in line:
                x.append(x1)
                x.append(x2)
                y.append(y1)
                y.append(y2)
        
        k, b = 0, 0
        if x and y:
            k, b, r, p, err = stats.linregress(x, y)
        k, b = self.approximate(k, b, history)
        
        bottom_y = self.region[0][1]
        left_line_middle_x = int((bottom_y - b) / k)
        middle_y = self.region[1][1]
        right_line_bottom_x = int((middle_y - b) / k)
        return [[[left_line_middle_x, bottom_y, right_line_bottom_x, middle_y]]]

    def clear_history(self):
        """ Clears the history """
        self.left_history.clear()
        self.right_history.clear()

Processing testing images:

In [4]:
test_images_dir = 'test_images/'
test_images_output_dir = 'test_images_output/'

for file in os.listdir(test_images_dir):
    img = mpimg.imread(test_images_dir + file)
    detector = LaneLinesDetector(region=[(0,539), (450, 325), (490, 325), (959,539)],
                                 stabilization_history_size=1, hough_rho=2)
    output = detector.pipeline(img)
    output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
    cv2.imwrite(test_images_output_dir + file, output)

## Test on Videos

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [5]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

Create the detector:

In [6]:
detector = LaneLinesDetector(region=[(0,539), (450, 325), (490, 325), (959,539)], hough_rho=2)

Let's try the one with the solid white lane on the right first ...

In [7]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(detector.pipeline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

t:   2%|███▌                                                                                                                                                            | 5/221 [00:00<00:04, 46.83it/s, now=None]

Moviepy - Building video test_videos_output/solidWhiteRight.mp4.
Moviepy - Writing video test_videos_output/solidWhiteRight.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/solidWhiteRight.mp4
Wall time: 4.17 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [8]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

Now for the one with the solid yellow lane on the left. This one's more tricky!

Clear the detector history after the previous video:

In [9]:
detector.clear_history()

Processs the video:

In [10]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(detector.pipeline)
%time yellow_clip.write_videofile(yellow_output, audio=False)

t:   1%|▉                                                                                                                                                               | 4/681 [00:00<00:17, 39.74it/s, now=None]

Moviepy - Building video test_videos_output/solidYellowLeft.mp4.
Moviepy - Writing video test_videos_output/solidYellowLeft.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/solidYellowLeft.mp4
Wall time: 11.4 s


In [11]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

Create a new detector. The last video has another resolution and the previous region doesn't match to the video:

In [12]:
detector_challenge = LaneLinesDetector(region=[(200,680), (600, 450), (750, 450), (1100,650)], hough_rho=2)

Process the video:

In [13]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(detector_challenge.pipeline)
%time challenge_clip.write_videofile(challenge_output, audio=False)

t:   1%|█▉                                                                                                                                                              | 3/251 [00:00<00:09, 27.22it/s, now=None]

Moviepy - Building video test_videos_output/challenge.mp4.
Moviepy - Writing video test_videos_output/challenge.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/challenge.mp4
Wall time: 7.47 s


In [14]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))